<a href="https://colab.research.google.com/github/deiteris/voice-changer/blob/master-custom/Colab_RealtimeVoiceChanger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [W-okada's Deiteris' Fork Realtime Voice Changer](https://github.com/deiteris/voice-changer) | **Google Colab**

---

## **READ ME - VERY IMPORTANT**

You can use the following settings for optimal results:

Best performance (with good quality): `f0: fcpe | Chunk: 80.0ms or higher | Extra: 5s`<br>
Best quality: `f0: rmvpe | Chunk: 112.0ms or higher | Extra: 5s`<br>
**Don't forget to select your Colab GPU in the GPU field (<b>Tesla T4</b>, for free users)*

You can tune `Chunk` for lower/higher delay and `Extra` for better quality.

<br><br>

---

### Always use Colab GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---

# **Credits and Support**
Original W-okada Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Original instructions by [Hina](https://github.com/HinaBl)<br>
HRZ Tunnel and Encryption by [Nick088](https://linktr.ee/Nick088)<br>

Need help? [AI Hub Discord](https://discord.gg/aihub) » ***#help-w-okada***

---

In [ ]:
#=================Updated=================
# @title **[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.**
import os
import time
from google.colab import drive

#@markdown ---
# @title **[Optional]** Connect to Google Drive
# @markdown Using Google Drive will automatically save your uploaded models for later use. Make sure you have sufficient amount of space on your Google Drive.

%cd /content

Use_Drive=False #@param {type:"boolean"}

!pip install colorama python-dotenv pyngrok --quiet

!npm install -g @hrzn/cli > /dev/null 2>&1

from colorama import Fore, Style
import requests


import codecs
LATEST_RELEASE = codecs.decode("uggcf://ncv.tvguho.pbz/ercbf/qrvgrevf/ibvpr-punatre/eryrnfrf/yngrfg", "rot13")
LINUX_PREBUILT = codecs.decode("ibvpr-punatre-yvahk-nzq64-phqn.gne.tm", "rot13")
REAL1 = codecs.decode("ibvpr-punatre", "rot13")
REAL2 = codecs.decode("ZZIPFreireFVB", "rot13")


print(f"{Fore.CYAN}> Downloading prebuilt executable...{Style.RESET_ALL}")

res = requests.get(LATEST_RELEASE)
release_info = res.json()

for asset in release_info['assets']:
   if not asset['name'].startswith(LINUX_PREBUILT):
      continue
   download_url = asset['browser_download_url']
   !wget -q --show-progress {download_url}

print(f"{Fore.GREEN}> Unpacking...{Style.RESET_ALL}")
!cat {LINUX_PREBUILT}.* | tar -xzf -
print(f"{Fore.GREEN}> Finished unpacking!{Style.RESET_ALL}")
!rm -rf {LINUX_PREBUILT}.*

!mv {REAL2} ML_Program
!mv ML_Program/{REAL2} ML_Program/ML_Program


%cd /content

if Use_Drive and not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

    !mkdir -p /content/drive/MyDrive/$REAL1/server/model_dir
    !mkdir -p /content/drive/MyDrive/$REAL1/server/pretrain


if Use_Drive and not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

  !mkdir -p /content/drive/MyDrive/$REAL1/server/model_dir
  !mkdir -p /content/drive/MyDrive/$REAL1/server/pretrain
  !rm -rf /content/$REAL2/server/model_dir
  !rm -rf /content/$REAL2/server/pretrain

  time.sleep(5)

  os.symlink(f"/content/drive/MyDrive/{REAL1}/server/model_dir", f"/content/ML_Program/model_dir", True)
  os.symlink(f"/content/drive/MyDrive/{REAL2}/server/pretrain", f"/content/ML_Program/pretrain", True)

print(f"{Fore.GREEN}> Successfully downloaded and unpacked the binary!{Style.RESET_ALL}")

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!apt-get -y install libportaudio2 -qq

In [ ]:
#=================Updated=================
# @title **[2]** Set server configuration
# @markdown This cell will set the server configuration.

%cd /content/ML_Program

from dotenv import set_key

set_key('.env', "SAMPLE_MODE", "")

Ready = True

In [ ]:
#=======================Updated=========================

# @title Start Server **using ngrok**
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a few minutes (usually ~1-2 minutes)

#======================Tunnels===========================

TUNNEL = "NGROK" #@param ["NGROK","HRZN"]

# @markdown ---
# @markdown You'll need a ngrok or hrzn account, but <font color=green>**it's free**</font> and easy to create!
# @markdown ---
# @markdown **1** - Create a <font color=green>**free**</font> account at [ngrok](https://dashboard.ngrok.com/signup) / [hrzn](https://hrzn.run/login) or **login with Google/Github account**\
# @markdown **2** - If you didn't logged in with Google/Github, you will need to **verify your e-mail**!\
# @markdown **3** - Get your [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) or [hrzn](https://hrzn.run/dashboard) to get your auth token, and place it here:
Token = 'NGROK | HRZN TOKEN' # @param {type:"string"}
# @markdown **4** - *(optional for ngrok)* Change to a region near to you or keep at United States if increase latency\
# @markdown `Default Region: ap - Asia/Pacific (Singapore)`
Region = "ap - Asia/Pacific (Singapore)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

#@markdown **5** - *(optional)* Other options:
ClearConsole = True  # @param {type:"boolean"}

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------

PORT = 18888

import json
import threading, time, socket
from IPython.display import clear_output


if not globals().get('Ready', False):
    print("Go back and run first and second cells.")
else:
  if TUNNEL == "NGROK":
    from pyngrok import conf, ngrok
    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    conf.get_default().authtoken = Token
    conf.get_default().region = Region
    conf.set_default(MyConfig);


    from pyngrok import ngrok

    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url
  elif TUNNEL == "HRZN":
    !rm -rf url.txt
    !hrzn login $Token
    os.system(f"hrzn tunnel http://localhost:{PORT} >> url.txt 2>&1 &")
    time.sleep(5)

    with open('url.txt', 'r') as file:
      public_url = file.read()
      public_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
      public_url = public_url[0]


  set_key('.env', "ALLOWED_ORIGINS", json.dumps([public_url]))


  def wait_for_server():
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', PORT))
          if result == 0:
              break
          sock.close()
      if ClearConsole:
          clear_output()
      print("--------- SERVER READY! ---------")
      print("Your server is available at:")
      print(public_url)
      print("---------------------------------")

  threading.Thread(target=wait_for_server, daemon=True).start()

  !./ML_Program

  if TUNNEL == "NGROK":
    ngrok.disconnect(ngrokConnection.public_url)
    print("--------- SERVER STOPPED! ---------")
  elif TUNNEL == "HRZN":
    !rm -rf url.txt
    !fuser -k ${PORT}
    print("--------- SERVER STOPPED! ---------")